In [1]:
!echo $XLA_FLAGS

/bin/bash: /home/padamnoo/.conda/envs/general/lib/libtinfo.so.6: no version information available (required by /bin/bash)
--xla_gpu_cuda_data_dir=/home/padamnoo/.conda/envs/general/


In [13]:
import os
import sys
import shutil
import argparse
import numpy as np
import tensorflow as tf
import scipy.io as sio
import glob
# sys.path.append('git/_framework')
from utils import util as U
from core.data_provider import DataProvider
from core.trainer_tf_uni import Trainer
from core.data_processor import SimpleImageProcessor
from core.learning_rate import StepDecayLearningRate
from model.model_wgan_uni import GANModel
from model.gan_10k import Generator, Discriminator
import timeit

In [14]:
start = timeit.default_timer()
epochs = 2
batch_size = 1
mini_batch_size = 1
eval_batch_size = 1

eval_frequency = 2
use_bn = False
use_res = True
learning_rate = 0.001
g_alpha = 40
g_lambda = 10
g_beta = 10
dropout = 0
resize = None
output_path = '/home/padamnoo/Desktop/Results/wgan_10k_640_zm_mm'
output_path = '{}_batch{}_bn{}_dp{}_a{}_la{}_lr{}'.format(output_path, batch_size, int(use_bn), dropout, g_alpha, g_lambda, learning_rate)
saved_filelists = 'std_sex_640_160_62.mat'

In [15]:
import platform

if platform.system() == 'Windows':
    data_path = './'
    output_path = 'results/' + output_path
if platform.system() == 'Linux':
    data_path = './'
    output_path = 'results/' + output_path

In [16]:
org_path = data_path + '/std96'
saved_filelists = data_path + '/' + saved_filelists
# load filenames
# file_mat = sio.loadmat(saved_filelists)

# load filenames
data_path = '/home/padamnoo/Desktop/Data/ModelDataPreprocessedResized/'
train_list_T1 = glob.glob(data_path + 'train/T1/*')
valid_list_T1 = glob.glob(data_path + 'valid/T1/*')
train_list_T2 =glob.glob(data_path + 'train/T2/*')
valid_list_T2 = glob.glob(data_path + 'valid/T2/*')
# test_list = glob.glob(data_path + '/test/*_img.png')

# set key for input images and labels
org_suffix = '_img.png'
lab_suffix = '_lab.png'

In [17]:
len(valid_list_T2) + len(valid_list_T1)

576

In [18]:
pre = {org_suffix: [ 'zero-mean', ('channelcheck', 1)],
       lab_suffix: [('one-hot', [0, 63, 127, 191, 255]), ('channelcheck', 5)]
}      

pre_ = {org_suffix: [ 'zero-mean', ('channelcheck', 1)],
        lab_suffix: [('one-hot', [0, 63, 127, 191, 255]), ('channelcheck', 5)]
       }

In [19]:
processor = SimpleImageProcessor(pre=pre)
processor_no_p = SimpleImageProcessor(pre=pre_)

In [20]:
train_provider_T1_t = DataProvider(train_list_T1, [org_suffix, lab_suffix],
                        is_pre_load=False,
                        is_shuffle=True,
                        # temp_dir=output_path,
                        processor=processor)

valid_provider_T1_t = DataProvider(valid_list_T1, [org_suffix, lab_suffix],
                        is_pre_load=False,
                        # temp_dir=output_path,
                        processor=processor)

train_provider_T1 = DataProvider(train_list_T1, [org_suffix],
                        is_pre_load=False,
                        is_shuffle=True,
                        # temp_dir=output_path,
                        processor=processor)

valid_provider_T1 = DataProvider(valid_list_T1, [org_suffix],
                        is_pre_load=False,
                        # temp_dir=output_path,
                        processor=processor)

train_provider_T2 = DataProvider(train_list_T2, [org_suffix],
                        is_pre_load=False,
                        is_shuffle=True,
                        # temp_dir=output_path,
                        processor=processor)

valid_provider_T2 = DataProvider(valid_list_T2, [org_suffix],
                        is_pre_load=False,
                        # temp_dir=output_path,
                        processor=processor)



train_provider_T1_t_w = DataProvider(train_list_T1, [org_suffix, lab_suffix],
                                   is_pre_load=False,
                                   is_shuffle=True,
                                   # temp_dir=output_path,
                                   processor=processor_no_p,
                                is_aug=False)

train_provider_T1_w = DataProvider(train_list_T1, [org_suffix],
                                 is_pre_load=False,
                                 is_shuffle=True,
                                 # temp_dir=output_path,
                                 processor=processor_no_p,
                                   is_aug=False)

train_provider_T2_w = DataProvider(train_list_T2, [org_suffix],
                                  is_pre_load=False,
                                  is_shuffle=True,
                                  # temp_dir=output_path,
                                  processor=processor_no_p,
                                    is_aug=False)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_all_data',
 '_build_temp_folder',
 '_cur_i',
 '_file_list',
 '_is_aug',
 '_is_shuffle',
 '_load_data',
 '_load_temp_file',
 '_next_idx',
 '_org_suffix',
 '_other_suffix',
 '_processor',
 '_temp_dir',
 'clipped_zoom',
 'rotation',
 'size']

In [11]:
gen = Generator(2, 4, 16, use_bn=use_bn)
disc = Discriminator(1, 4, 16, use_bn=use_bn)
model = GANModel([gen, disc], org_suffix, lab_suffix, g_alpha=g_alpha, g_lambda=g_lambda, g_beta=g_beta, dropout=0)
gen_lr = StepDecayLearningRate(learning_rate=learning_rate,
                           decay_step=10,
                           decay_rate=0.8,
                           data_size=train_provider_T1.size + train_provider_T2.size,
                           batch_size=batch_size)
disc_lr = StepDecayLearningRate(learning_rate=learning_rate,
                           decay_step=10,
                           decay_rate=0.8,
                           data_size=train_provider_T1.size + train_provider_T2.size,
                           batch_size=batch_size)
gen_optimizer = tf.keras.optimizers.Adam(gen_lr)
disc_optimizer = tf.keras.optimizers.Adam(disc_lr)
trainer = Trainer(model)

2023-02-10 16:20:04.007198: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 16:20:07.203306: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22025 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1b:00.0, compute capability: 8.6
2023-02-10 16:20:07.204049: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22279 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1c:00.0, compute capability: 8.6
2023-02-10 16:20:07.204516: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localho

In [12]:
# train
results = trainer.train(train_provider_T1, train_provider_T2, valid_provider_T1, valid_provider_T2, train_provider_T1_t, valid_provider_T1_t,
                        train_provider_T1_w, train_provider_T2_w,train_provider_T1_t_w,
                        epochs=epochs,
                       batch_size=batch_size,
                       mini_batch_size=mini_batch_size,
                       output_path=output_path,
                       optimizer=[gen_optimizer, disc_optimizer],
                       learning_rate=[gen_lr, disc_lr],
                       eval_frequency=eval_frequency)

Start training: epochs 2, learning rate ['0.001', '0.001'], batch size 1, mini-batch size 1, training data 2584, validation data 576.
[[[[ True]
   [ True]
   [ True]
   ...
   [ True]
   [ True]
   [ True]]

  [[ True]
   [ True]
   [ True]
   ...
   [ True]
   [ True]
   [ True]]

  [[ True]
   [ True]
   [ True]
   ...
   [ True]
   [ True]
   [ True]]

  ...

  [[ True]
   [ True]
   [ True]
   ...
   [ True]
   [ True]
   [ True]]

  [[ True]
   [ True]
   [ True]
   ...
   [ True]
   [ True]
   [ True]]

  [[ True]
   [ True]
   [ True]
   ...
   [ True]
   [ True]
   [ True]]]


 [[[ True]
   [ True]
   [ True]
   ...
   [ True]
   [ True]
   [ True]]

  [[ True]
   [ True]
   [ True]
   ...
   [ True]
   [ True]
   [ True]]

  [[ True]
   [ True]
   [ True]
   ...
   [ True]
   [ True]
   [ True]]

  ...

  [[ True]
   [ True]
   [ True]
   ...
   [ True]
   [ True]
   [ True]]

  [[ True]
   [ True]
   [ True]
   ...
   [ True]
   [ True]
   [ True]]

  [[ True]
   [ True]
   

2023-02-10 16:20:10.574155: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-02-10 16:20:17.033768: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x53061d90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-10 16:20:17.033819: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-02-10 16:20:17.033832: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-02-10 16:20:17.033842: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (2): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-02-10 16:20:17.033851: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (3): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-02-10 16:20:17.033860: I tensorflow/compiler/xla/service/service.cc:

KeyboardInterrupt: 

In [ ]:
tf.device('gpu')